In [1]:
import pandas as pd
import numpy as np

## multi-LABELS until new annoations from CW come out

1. see which columns are shared between cw and kaggle (mostly comp and desc)
2. see common rows and look into how much kaggle can augment cw (pretty well) 
3. extract out kaggle labels in rows as columns via 1-hot-encode
4. if kaggle does not augment, then drop shared rows
5. see if addition of kaggle annotations will cause further class imbalance
6. merge repeated columns
7. make sure data is 0/1
8. make sure each df has parallel columns
9. union between df's
10. should have around 1600 new sample to create model

In [23]:
cw = pd.read_csv("cwclasses.csv")
list(cw)

['paper_id',
 'paper_txt',
 'sha',
 'in silico',
 'in vitro',
 'in vivo',
 'Systematic review or meta-analysis of RCTs',
 'RCT',
 'Non-randomized controlled trial',
 'Comparative study',
 'Descriptive study',
 'Systematic review or meta-analysis of studies other than RCTs',
 'Other ']

In [44]:
kg = pd.read_csv("kaggleclasses.csv")

list(kg)

['paper_id', 'paper_txt', 'label', 'id']

In [35]:
shared = cw.merge(kg, how='inner', left_on='paper_id', right_on='paper_id')
shared

,paper_id,paper_txt_x,sha,in silico,in vitro,in vivo,Systematic review or meta-analysis of RCTs,RCT,Non-randomized controlled trial,Comparative study,Descriptive study,Systematic review or meta-analysis of studies other than RCTs,Other,paper_txt_y,label,id
0,4bc6ead9368ec84dba7c56b78624f681293e568b,trend traumatic injury child visit emergency d...,4bc6ead9368ec84dba7c56b78624f681293e568b,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0,Characteristics and trends of traumatic injuri...,7,4bc6ead9368ec84dba7c56b78624f681293e568b
1,5a7988cd92e05a8bca504b5f1a467e2c8d3976d2,analysis transmission control mainland base ma...,5a7988cd92e05a8bca504b5f1a467e2c8d3976d2,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,Analysis of Transmission and Control of Tuberc...,10,5a7988cd92e05a8bca504b5f1a467e2c8d3976d2
2,5b15c62ee761d6eefeb7d2ef023f804bfdc41edc,optimal decision model hospital vacant school ...,5b15c62ee761d6eefeb7d2ef023f804bfdc41edc,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0,Optimal Decision Model for Sustainable Hospita...,9,5b15c62ee761d6eefeb7d2ef023f804bfdc41edc
3,5e9ccecdd40825f5c30da3a900fc5cf1b063b47d,infectious disease role tnf gene polymorphism ...,5e9ccecdd40825f5c30da3a900fc5cf1b063b47d,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,BMC Infectious Diseases Roles of TNF-α gene po...,8,5e9ccecdd40825f5c30da3a900fc5cf1b063b47d
4,766e0afdd0d3c2a1f8d9a6c8b318ca401db0a8bd,objective time become classic public health in...,766e0afdd0d3c2a1f8d9a6c8b318ca401db0a8bd,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,c Ann Arbor Veterans Affairs Hospital Objectiv...,9,766e0afdd0d3c2a1f8d9a6c8b318ca401db0a8bd
5,7ebedf373e4b3c414333bdd7be729cfbcab489b3,high reproduction number mathematical model ma...,7ebedf373e4b3c414333bdd7be729cfbcab489b3,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,High reproduction number of Middle East respir...,10,7ebedf373e4b3c414333bdd7be729cfbcab489b3
6,d7d62e768f4a8eb40ba522eca28e173e5d53cbbc,infection rheumatoid arthritis close new type ...,d7d62e768f4a8eb40ba522eca28e173e5d53cbbc,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1,COVID-19 infection and rheumatoid arthritis: F...,4,d7d62e768f4a8eb40ba522eca28e173e5d53cbbc
7,eac85c85c3a2ac1d3fe53e1e3b3fee98f52527ad,evaluation preparedness student volunteer seni...,eac85c85c3a2ac1d3fe53e1e3b3fee98f52527ad,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0,Evaluation of preparedness of healthcare stude...,7,eac85c85c3a2ac1d3fe53e1e3b3fee98f52527ad


In [45]:
#now need to get a better idea of how kg will augment cw by mapping kg classes to cw classes (multilabel to multiclass)
kg.explode('label')
kg['label'].unique()

array([ 5,  9, 10,  7,  2,  0,  4,  8,  1,  6,  3])

In [46]:
dic_map={
         1:'Systematic reivew or meta-analysis of RCTs',
         2:"RCT",
         3:"Non-randomized controlled trial",
         4:'Comparative study ',
         5:'Comparative study',
         6:'Comparative study',
         7:'Descriptive study',
         8:'Comparative study',
         9:'Descriptive study',
         10:"in silico",
         0:'Other'
        }

In [47]:
kg['mapped'] = kg.apply(lambda x: dic_map[x.label], axis = 1) 
kg.head()

,paper_id,paper_txt,label,id,mapped
0,001d8d54a7e73e761f779c81661595cc5ae2ca08,Intervention time series analysis of crime rat...,5,001d8d54a7e73e761f779c81661595cc5ae2ca08,Comparative study
1,0043d044273b8eb1585d3a66061e9b4e03edc062,Evaluation of the tuberculosis programme in Ni...,9,0043d044273b8eb1585d3a66061e9b4e03edc062,Descriptive study
2,00911cf4f99a3d5ae5e5b787675646a743574496,CHEER: hierarCHical taxonomic classification f...,10,00911cf4f99a3d5ae5e5b787675646a743574496,in silico
3,00951716e01c8e0cc341770389fc38d1b5455210,"Knowledge of, attitudes toward, and preventive...",7,00951716e01c8e0cc341770389fc38d1b5455210,Descriptive study
4,012debf5a240a496518af146ddfc16c958339c2b,Preparedness and response against diseases wit...,9,012debf5a240a496518af146ddfc16c958339c2b,Descriptive study


In [48]:
shared2 = kg.merge(cw, how='inner', left_on='paper_id', right_on='paper_id')
shared2

,paper_id,paper_txt_x,label,id,mapped,paper_txt_y,sha,in silico,in vitro,in vivo,Systematic review or meta-analysis of RCTs,RCT,Non-randomized controlled trial,Comparative study,Descriptive study,Systematic review or meta-analysis of studies other than RCTs,Other
0,4bc6ead9368ec84dba7c56b78624f681293e568b,Characteristics and trends of traumatic injuri...,7,4bc6ead9368ec84dba7c56b78624f681293e568b,Descriptive study,trend traumatic injury child visit emergency d...,4bc6ead9368ec84dba7c56b78624f681293e568b,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0
1,5a7988cd92e05a8bca504b5f1a467e2c8d3976d2,Analysis of Transmission and Control of Tuberc...,10,5a7988cd92e05a8bca504b5f1a467e2c8d3976d2,in silico,analysis transmission control mainland base ma...,5a7988cd92e05a8bca504b5f1a467e2c8d3976d2,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0
2,5b15c62ee761d6eefeb7d2ef023f804bfdc41edc,Optimal Decision Model for Sustainable Hospita...,9,5b15c62ee761d6eefeb7d2ef023f804bfdc41edc,Descriptive study,optimal decision model hospital vacant school ...,5b15c62ee761d6eefeb7d2ef023f804bfdc41edc,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0
3,5e9ccecdd40825f5c30da3a900fc5cf1b063b47d,BMC Infectious Diseases Roles of TNF-α gene po...,8,5e9ccecdd40825f5c30da3a900fc5cf1b063b47d,Comparative study,infectious disease role tnf gene polymorphism ...,5e9ccecdd40825f5c30da3a900fc5cf1b063b47d,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0
4,766e0afdd0d3c2a1f8d9a6c8b318ca401db0a8bd,c Ann Arbor Veterans Affairs Hospital Objectiv...,9,766e0afdd0d3c2a1f8d9a6c8b318ca401db0a8bd,Descriptive study,objective time become classic public health in...,766e0afdd0d3c2a1f8d9a6c8b318ca401db0a8bd,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0
5,7ebedf373e4b3c414333bdd7be729cfbcab489b3,High reproduction number of Middle East respir...,10,7ebedf373e4b3c414333bdd7be729cfbcab489b3,in silico,high reproduction number mathematical model ma...,7ebedf373e4b3c414333bdd7be729cfbcab489b3,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0
6,d7d62e768f4a8eb40ba522eca28e173e5d53cbbc,COVID-19 infection and rheumatoid arthritis: F...,4,d7d62e768f4a8eb40ba522eca28e173e5d53cbbc,Comparative study,infection rheumatoid arthritis close new type ...,d7d62e768f4a8eb40ba522eca28e173e5d53cbbc,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1
7,eac85c85c3a2ac1d3fe53e1e3b3fee98f52527ad,Evaluation of preparedness of healthcare stude...,7,eac85c85c3a2ac1d3fe53e1e3b3fee98f52527ad,Descriptive study,evaluation preparedness student volunteer seni...,eac85c85c3a2ac1d3fe53e1e3b3fee98f52527ad,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0


In [50]:
list(shared2)

['paper_id',
 'paper_txt_x',
 'label',
 'id',
 'mapped',
 'paper_txt_y',
 'sha',
 'in silico',
 'in vitro',
 'in vivo',
 'Systematic review or meta-analysis of RCTs',
 'RCT',
 'Non-randomized controlled trial',
 'Comparative study',
 'Descriptive study',
 'Systematic review or meta-analysis of studies other than RCTs',
 'Other ']

In [52]:
shared2.drop(columns=['paper_id',
                      'paper_txt_x',
                      'label',
                      'id',
                      'paper_txt_y',
                      'sha'])

,mapped,in silico,in vitro,in vivo,Systematic review or meta-analysis of RCTs,RCT,Non-randomized controlled trial,Comparative study,Descriptive study,Systematic review or meta-analysis of studies other than RCTs,Other
0,Descriptive study,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0
1,in silico,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0
2,Descriptive study,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0
3,Comparative study,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0
4,Descriptive study,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0
5,in silico,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0
6,Comparative study,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1
7,Descriptive study,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0


Based on this small sample, looks like kg annotations would augment cw annotations since 4 pub's would have more labels than presented if just left with cw annotations. (50%) This would greatly enhance annotations dataset.

(Before proceeding to one-hot-encoding, need to take out shared pub id's from cw and kg since shared will be added to union between cw and kg at the end.)

In [53]:
# #extract out kaggle labels in rows as columns via 1-hot-encode

# one_hot = pd.get_dummies(kg['mapped'])
# #drop mapped column as it is encoded now
# kg = kg.drop('mapped',axis = 1)
# # Join the encoded mapped to df3
# kg = kg.join(one_hot)
# kg